In [128]:
import pandas as pd
import numpy as np
import re

In [129]:
yearsToCheck = list((np.arange(2000, 2020, 1)).astype(str))
developedCountries = ['Australia','Japan','Sweden','Switzerland','USA']
developingCountries = ['Brazil','Egypt','Greece','India','South Africa']
underdevelopedCountries = ['Afghanistan','Bangladesh','Cambodia','Ethiopia','Senegal']

In [181]:
populationCsvPath = 'data/raw_data/pop.csv'
readPopCsv = pd.read_csv(populationCsvPath)
populationDataFrame = (pd.DataFrame(readPopCsv).set_index('country'))[yearsToCheck]

developed_populationDataFrame = populationDataFrame.loc[developedCountries]
developing_populationDataFrame = populationDataFrame.loc[developingCountries]
underdeveloped_populationDataFrame = populationDataFrame.loc[underdevelopedCountries]

df = [developed_populationDataFrame, underdeveloped_populationDataFrame, developing_populationDataFrame]

for j in df:
    for i in yearsToCheck:
        for index in range(len(j[i])):
            try:
                if (j[i][index][-1] == 'M') & ('.' not in j[i][index]):
                    j[i][index] = j[i][index].replace('M', '000000')
                elif (j[i][index][-1] == 'B') & ('.' not in j[i][index]):
                    j[i][index] = j[i][index].replace('B', '000000000')
                elif (j[i][index][-1] == 'M') & (j[i][index][-3] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('M', '00000')
                elif (j[i][index][-1] == 'B') & (j[i][index][-3] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('B', '00000000')
                elif (j[i][index][-1] == 'M') & (j[i][index][-4] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('M', '0000')
                elif (j[i][index][-1] == 'B') & (j[i][index][-4] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('B', '0000000')

            except:
                print(j[i][index])



developed_populationDataFrame.to_csv('data/clean_data/population/csv/developed_population.csv')
developing_populationDataFrame.to_csv('data/clean_data/population/csv/developing_population.csv')
underdeveloped_populationDataFrame.to_csv('data/clean_data/population/csv/underveloped_population.csv')

developed_populationDataFrame.to_json('data/clean_data/population/json/developed_population.json', indent = 1)
developing_populationDataFrame.to_json('data/clean_data/population/json/developing_population.json', indent = 1)
underdeveloped_populationDataFrame.to_json('data/clean_data/population/json/underveloped_population.json', indent = 1)

In [117]:
gdpUSDPath = 'data/raw_data/total_gdp_us_inflation_adjusted.csv'
readGdpUSDCsv = pd.read_csv(gdpUSDPath)
GdpDataFrame = (pd.DataFrame(readGdpUSDCsv).set_index('country'))[yearsToCheck]

developed_GdpDataFrame = GdpDataFrame.loc[developedCountries]
developing_GdpDataFrame = GdpDataFrame.loc[developingCountries]
underdeveloped_GdpDataFrame = GdpDataFrame.loc[underdevelopedCountries]

developed_GdpDataFrame.to_csv('data/clean_data/gdpUSD/csv/developed_gdp.csv')
developing_GdpDataFrame.to_csv('data/clean_data/gdpUSD/csv/developing_gdp.csv')
underdeveloped_GdpDataFrame.to_csv('data/clean_data/gdpUSD/csv/underdeveloped_gdp.csv')

developed_GdpDataFrame.to_json('data/clean_data/gdpUSD/json/developed_gdp.json', indent = 1)
developing_GdpDataFrame.to_json('data/clean_data/gdpUSD/json/developing_gdp.json', indent = 1)
underdeveloped_GdpDataFrame.to_json('data/clean_data/gdpUSD/json/underdeveloped_gdp.json', indent = 1)

In [118]:
avgDailyIncPpPath = 'data/raw_data/mincpcap_cppp.csv'
readvgDailyIncCsv = pd.read_csv(avgDailyIncPpPath)
avgDailyIncPpDataFrame = (pd.DataFrame(readvgDailyIncCsv).set_index('country'))[yearsToCheck]

developed_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[developedCountries]
developing_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[developingCountries]
underdeveloped_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[underdevelopedCountries]

developed_avgDailyIncPpDataFrame.to_csv('data/clean_data/averageIncomePerPerson/csv/developed_average_daily_income.csv')
developing_avgDailyIncPpDataFrame.to_csv('data/clean_data/averageIncomePerPerson/csv/developing_average_daily_income.csv')
underdeveloped_avgDailyIncPpDataFrame.to_csv('data/clean_data/averageIncomePerPerson/csv/underdeveloped_average_daily_income.csv')

developed_avgDailyIncPpDataFrame.to_json('data/clean_data/averageIncomePerPerson/json/developed_average_daily_income.json', indent = 1)
developing_avgDailyIncPpDataFrame.to_json('data/clean_data/averageIncomePerPerson/json/developing_average_daily_income.json', indent = 1)
underdeveloped_avgDailyIncPpDataFrame.to_json('data/clean_data/averageIncomePerPerson/json/underdeveloped_average_daily_income.json', indent = 1)

In [119]:
# This is in tonnes
co2PerCapPath = 'data/raw_data/co2_pcap_cons.csv'
readco2PerCapCsv = pd.read_csv(co2PerCapPath)
co2PerCapDataFrame = (pd.DataFrame(readco2PerCapCsv).set_index('country'))[yearsToCheck]

developed_co2PerCapDataFrame = co2PerCapDataFrame.loc[developedCountries]
developing_co2PerCapDataFrame = co2PerCapDataFrame.loc[developingCountries]
underdeveloped_co2PerCapDataFrame = co2PerCapDataFrame.loc[underdevelopedCountries]

developed_co2PerCapDataFrame.to_csv('data/clean_data/co2PerCapitaConsumption/csv/developed_co2_per_capita.csv')
developing_co2PerCapDataFrame.to_csv('data/clean_data/co2PerCapitaConsumption/csv/developing_co2_per_capita.csv')
underdeveloped_co2PerCapDataFrame.to_csv('data/clean_data/co2PerCapitaConsumption/csv/underdeveloped_co2_per_capita.csv')

developed_co2PerCapDataFrame.to_json('data/clean_data/co2PerCapitaConsumption/json/developed_co2_per_capita.json', indent = 1)
developing_co2PerCapDataFrame.to_json('data/clean_data/co2PerCapitaConsumption/json/developing_co2_per_capita.json', indent = 1)
underdeveloped_co2PerCapDataFrame.to_json('data/clean_data/co2PerCapitaConsumption/json/underdeveloped_co2_per_capita.json', indent = 1)

In [120]:
# This is in million tonnes
co2TotalEmissionsPath = 'data/raw_data/co2_cons.csv'
readco2TotalEmissionCsv = pd.read_csv(co2TotalEmissionsPath)
co2TotalEmissionsDataFrame = (pd.DataFrame(readco2TotalEmissionCsv).set_index('country'))[yearsToCheck]

developed_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[developedCountries]
developing_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[developingCountries]
underdeveloped_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[underdevelopedCountries]

developed_co2TotalEmissionsDataFrame.to_csv('data/clean_data/co2EmissionsTotal/csv/developed_co2_total_emissions.csv')
developing_co2TotalEmissionsDataFrame.to_csv('data/clean_data/co2EmissionsTotal/csv/developing_co2_total_emissions.csv')
underdeveloped_co2TotalEmissionsDataFrame.to_csv('data/clean_data/co2EmissionsTotal/csv/underdeveloped_co2_total_emissions.csv')

developed_co2TotalEmissionsDataFrame.to_json('data/clean_data/co2EmissionsTotal/json/developed_co2_total_emissions.json', indent = 1)
developing_co2TotalEmissionsDataFrame.to_json('data/clean_data/co2EmissionsTotal/json/developing_co2_total_emissions.json', indent = 1)
underdeveloped_co2TotalEmissionsDataFrame.to_json('data/clean_data/co2EmissionsTotal/json/underdeveloped_co2_total_emissions.json', indent = 1)